In [1]:
import pandas as pd

In [2]:
nutrition_df = pd.read_excel('https://query.data.world/s/3yg7x3cesk6lgbtepnezfdh2m36bui')

In [3]:
df = pd.read_csv('app/data/data.csv')

In [4]:
df['Ingredients']

0        ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...
1        ['2 large egg whites', '1 pound new potatoes (...
2        ['1 cup evaporated milk', '1 cup whole milk', ...
3        ['1 (¾- to 1-pound) round Italian loaf, cut in...
4        ['1 teaspoon dark brown sugar', '1 teaspoon ho...
                               ...                        
13496    ['1 cup all-purpose flour', '2/3 cup unsweeten...
13497    ['1 preserved lemon', '1 1/2 pound butternut s...
13498    ['Leftover katsuo bushi (dried bonito flakes) ...
13499    ['1 stick (1/2 cup) plus 1 tablespoon unsalted...
13500    ['12 medium to large fresh poblano chiles (2 1...
Name: Ingredients, Length: 13501, dtype: object

In [5]:
ingredients = eval(df['Ingredients'].iloc[0])

In [6]:
ingredients

['1 (3½–4-lb.) whole chicken',
 '2¾ tsp. kosher salt, divided, plus more',
 '2 small acorn squash (about 3 lb. total)',
 '2 Tbsp. finely chopped sage',
 '1 Tbsp. finely chopped rosemary',
 '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
 '¼ tsp. ground allspice',
 'Pinch of crushed red pepper flakes',
 'Freshly ground black pepper',
 '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
 '2 Tbsp. extra-virgin olive oil',
 '½ small red onion, thinly sliced',
 '3 Tbsp. apple cider vinegar',
 '1 Tbsp. white miso',
 '¼ cup all-purpose flour',
 '2 Tbsp. unsalted butter, room temperature',
 '¼ cup dry white wine',
 '2 cups unsalted chicken broth',
 '2 tsp. white miso',
 'Kosher salt, freshly ground pepper']

In [115]:
def cleanup(ingredients: list[str]):
    res = []
    for s in ingredients:
        s = s.lower()
        if s.count('-'):
            s = s[::-1].replace('-', ' ')[::-1]
        s = s.replace('lb.', 'lb')
        s = s.replace('gs', 'g')
        s = s.replace('lb', 'pound')
        s = s.replace('½', '.5')
        s = s.replace('¾', '.75')
        s = s.replace('¼', '.25')
        s = s.replace('cup', '128g')
        s = s.replace('tsp.', 'tablespoon')
        s = s.replace('tbsp.', 'tablespoon')
        s = s.replace('tablespoon', '21.25g')
        s = s.replace('teaspoon', '5.7g')
        s = s.replace('pound', '453.6g')
        s = s.replace('oz.', '28g')
        s = s.replace('⅓', '0.33')
        s = s.replace('loaf', '400g')
        s = s.replace('pinch', '0.35g')
        res.append(s)
    return res

In [117]:
def parse_grams(s):
    words = s.split(' ')
    if 'g' in words[0]:
        try: 
            grams = float(words[0].replace('g', ''))
            return grams
        except:
            pass
    for i in range(len(words) - 1):
        w0 = words[i]
        w1 = words[i + 1]
        if 'g' in w1:
            try:
                if '–' in w0:
                    w00, w01 = w0.split('–')
                    w00 = w00.replace('(', '')
                    w01 = w01.replace(')', '')
                    try:
                        w00 = float(w00)
                        w01 = float(w01)
                        w0 = (w01 + w00) / 2
                    except:
                        pass
                if type(w0) is str:
                    w0 = w0.replace('(', '')
                w1 = w1.replace(')', '')
                w0 = float(w0)
                w1 = w1.replace('g', '')
                w1 = float(w1)
                grams = w0 * w1
                return round(grams, 2)
            except: 
                continue

In [118]:
clean = cleanup(ingredients)

### 'gs' is weird

In [119]:
for s in clean:
    print(f'{s:100} {parse_grams(s)}')

1 (3.5–4 453.6g) whole chicken                                                                       1701.0
2.75 21.25g kosher salt, divided, plus more                                                          58.44
2 small acorn squash (about 3 453.6g total)                                                          1360.8
2 21.25g finely chopped sage                                                                         42.5
1 21.25g finely chopped rosemary                                                                     21.25
6 21.25g unsalted butter, melted, plus 3 21.25g room temperature                                     127.5
.25 21.25g ground allspice                                                                           5.31
0.35g of crushed red pepper flakes                                                                   0.35
freshly ground black pepper                                                                          None
0.33 400g good quality sturdy white bre

### to match words can use a special big list that only has "food" words in it

as a result shall obtain qty of grams and the product name to be matched in the nutrition df